
# **数据挖掘——Home Credit Default Risk**

Authors：李林（3120220938）、杨洋（3220211141）、敬甲男（3220221052）、李翰杰（3120220936）

github地址：https://github.com/leealim/kaggle-Home-Credit-Default-Risk

---


In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

version='v0.1'
app_tr_path="..\\data\\home-credit-default-risk\\application_train.csv"
app_te_path="..\\data\\home-credit-default-risk\\application_test.csv"

app_train = pd.read_csv(app_tr_path)
app_test = pd.read_csv(app_te_path)

In [124]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns.sort_values(
    '% of Total Values', ascending=False).round(1)
    return mis_val_table_ren_columns

missing_values = missing_values_table(app_train)
missing_values.head()

,Missing Values,% of Total Values
COMMONAREA_MEDI,214865,69.9
COMMONAREA_AVG,214865,69.9
COMMONAREA_MODE,214865,69.9
NONLIVINGAPARTMENTS_MODE,213514,69.4
NONLIVINGAPARTMENTS_AVG,213514,69.4


In [125]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le_count = 0
for col in app_train:
    if app_train[col].dtype == 'object':
        if len(list(app_train[col].unique())) <= 2:
            le.fit(app_train[col])
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            le_count += 1

app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

train_labels = app_train['TARGET']
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
app_train['TARGET'] = train_labels

In [126]:
def agg_numeric(df, parent_var, df_name):
    
    for col in df:
        if col != parent_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    parent_ids = df[parent_var].copy()
    numeric_df = df.select_dtypes('number').copy()
    numeric_df[parent_var] = parent_ids
    agg = numeric_df.groupby(parent_var).agg(['count', 'mean', 'max', 'min', 'sum'])

    columns = []
    for var in agg.columns.levels[0]:
        if var != parent_var:
            for stat in agg.columns.levels[1]:
                columns.append('%s_%s_%s' % (df_name, var, stat))
    agg.columns = columns
    
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg

def agg_categorical(df, parent_var, df_name):
    
    categorical = pd.get_dummies(df.select_dtypes('category'))
    categorical[parent_var] = df[parent_var]
    categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
    
    column_names = []
    for var in categorical.columns.levels[0]:
        for stat in ['sum', 'count', 'mean']:
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    categorical.columns = column_names
    
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

In [127]:
def kde_target(var_name, df):
    
    corr = df['TARGET'].corr(df[var_name])
    avg_repaid = df.loc[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.loc[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    sns.kdeplot(df.loc[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.loc[df['TARGET'] == 1, var_name], label = 'TARGET == 1')

    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend() 
    
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

In [128]:
import sys

def return_size(df):
    return round(sys.getsizeof(df) / 1e9, 2)

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    for c in df:
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df


In [129]:
previous = pd.read_csv('../data/home-credit-default-risk/previous_application.csv')
previous = convert_types(previous, print_info=True)
previous_agg = agg_numeric(previous, 'SK_ID_CURR', 'previous')
previous_counts = agg_categorical(previous, 'SK_ID_CURR', 'previous')

train = app_train
train = convert_types(train)
test = app_test
test = convert_types(test)

train = train.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
train = train.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

test = test.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
test = test.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

gc.enable()
del previous, previous_agg, previous_counts
gc.collect()

Original Memory Usage: 0.49 gb.
New Memory Usage: 0.18 gb.


2919

In [130]:
train.shape

(307511, 605)

In [131]:
def remove_missing_columns(train, test, threshold = 75):

    train_miss = train.isnull().sum()
    test_miss = test.isnull().sum()

    train_index_list = train_miss.index.tolist()
    missing_train_columns=[]
    for i in range(train_miss.shape[0]) :
        if 100 * train_miss[i] / len(train)>75:
            missing_train_columns.append(train_index_list[i])

    test_index_list = test_miss.index.tolist()
    missing_test_columns=[]
    for i in range(test_miss.shape[0]) :
        if 100 * test_miss[i] / len(test)>75:
            missing_test_columns.append(test_index_list[i])
    
    missing_columns = list(set(missing_train_columns + missing_test_columns))

    train = train.drop(columns = missing_columns)
    test = test.drop(columns = missing_columns)
    
    return train, test

train, test = remove_missing_columns(train, test)
train.shape

(307511, 599)

In [132]:
threshold = 0.9

corr_matrix = train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

train = train.drop(columns = to_drop)
test = test.drop(columns = to_drop)

In [133]:
train.shape

(307511, 508)

In [134]:

import xgboost
train_t=train.copy()
train_t = train_t.drop(columns = ['TARGET'])
x = train_t.values
y = train['TARGET'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2 , random_state = 5)
model = xgboost.XGBClassifier(n_estimatores = 200, max_depth = 8, subsample = 0.8, colsample_bytree = 0.8, 
                              min_child_weight = 50, random_state=27).fit(x_train, y_train)
#生成train数据放进模型后的答案
ans = model.predict_proba(x_test)

ans_test = model.predict_proba(test.values)
print(ans_test)



[10:52:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "n_estimatores" } are not used.

[[0.9538178  0.04618224]
 [0.86457473 0.13542528]
 [0.97555155 0.02444843]
 ...
 [0.9697569  0.03024312]
 [0.9441286  0.05587142]
 [0.7804461  0.21955387]]


In [135]:
#生成答案
index=test['SK_ID_CURR']
ans=pd.DataFrame(ans_test[:,1],columns = ['TARGET'])
result=pd.concat([index,ans],axis=1)
result.to_csv('../result/'+version+'/submission_t.csv',encoding = 'utf-8',index = 0)
result.shape

(48744, 2)

In [ ]:
app_te = pd.read_csv(app_te_path)

t = missing_values_table(app_te, "application_{train|test}.csv")
t_small=t.loc[t["% of Total Values"]<3]
t_large=t.loc[t["% of Total Values"]>3]
app_te.dropna(subset=t_small["Row"],
          axis=0, # axis=0表示删除行；
          how='any', # how=any表示若列name、age中，任意一个出现空值，就删掉该行
          inplace=True # inplace=True表示在原df上进行修改；
          )
app_te = app_te.reset_index(drop=True)

t = missing_values_table(app_te, "application_{train|test}.csv")
temp=t.drop(columns=["Special"]).isnull().T.any()
temp.loc[temp==True].index
rows=t.loc[temp.loc[temp==True].index].Row
for col in rows: 
    app_te[str(col)] = app_te[str(col)].fillna(value="MyNull")

t = missing_values_table(app_te, "application_{train|test}.csv")
t_house=t.loc[t["Description"]=="Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor"]
temp=app_te.loc[:,t_house["Row"].tolist()].isnull().sum(axis=1)
app_te["MY_HOUSE_OWN"]=temp
app_te_drop_list=t_house["Row"].tolist()
for col in app_te_drop_list: 
    app_te[col] = app_te[col].fillna(value=0)